Forecast for dixie sales
---

Forecast A:
Forecast Net Exchange per customer, per month,
with a forecast horrizon of 12 months

Forecast B:
Forecast Net Qty per Functional Category, Build type, and month,
with a forecast horizon of 12 months

In [3]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pandas.plotting import autocorrelation_plot
import datetime as dt
from dotenv import load_dotenv
load_dotenv()

# Import scripts after loading evnironment to check for refresh flag
import scripts

In [4]:
# Get data from SQL (or cache)
invoices = scripts.Invoices.load(refresh=True)
accounts = scripts.Accounts.load(refresh=True)

Loading invoices data from SQL...
Loading accounts data from SQL...


In [8]:
account = accounts.account_number[0]
sales_df = invoices[invoices.account_number == account].resample('M').sum().filter(['NetExchange'])

In [13]:
 # Calculate split point and split data
split_index = int(len(sales_df) * 0.9)
df_train, df_test = sales_df[:split_index], sales_df[split_index:]
X_train = df_train.index.map(dt.datetime.toordinal).values.reshape(-1, 1)
y_train = df_train.NetExchange.values.reshape(-1, 1)
X_test = df_test.index.map(dt.datetime.toordinal).values.reshape(-1, 1)
y_test = df_test.NetExchange.values.reshape(-1, 1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape    

((74, 1), (74, 1), (9, 1), (9, 1))

In [15]:
#* Fit a linear model using sklearn
lin_mdl = linear_model.LinearRegression()
lin_mdl.fit(X_train, y_train)
y_guess = lin_mdl.predict(X_test)